In [1]:
from unsloth import FastLanguageModel, is_bf16_supported
import torch
from unsloth.chat_templates import get_chat_template

max_seq_length = 4096
lora_rank = 64
dtype = None

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./models/unsloth-Qwen3-0.6B",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    dtype = dtype
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = False,
    loftq_config = None    
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/anthony/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA RTX 4500 Ada Generation. Num GPUs = 1. Max memory: 23.994 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.6.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [2]:
print(tokenizer.chat_template)

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0].role == 'system' %}
        {{- messages[0].content + '\n\n' }}
    {%- endif %}
    {{- "# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0].role == 'system' %}
        {{- '<|im_start|>system\n' + messages[0].content + '<|im_end|>\n' }}
    {%- endif %}
{%- endif %}
{%- set ns = namespace(multi_step_tool=true, last_query_index=messages|length - 1) %}
{%- for forward_message in messages %}
    {%- set index = (messages|length 

In [3]:
#from transformers import AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained("./models/Qwen2.5-0.5B")

# 2. Definir manualmente el chat_template con soporte para contextos

## Contextual Etiqueta
new_tokens = ["<|im_start|>context", "<|context|>"]
tokenizer.add_tokens(new_tokens)
model.resize_token_embeddings(len(tokenizer))

token_id = tokenizer.convert_tokens_to_ids("<|im_start|>context")
print(model.get_input_embeddings().weight[token_id])

custom_chat_template = """{%- set last_user_index = -1 %}
{%- for i in range(messages | length) %}
    {%- if messages[i]['role'] == 'user' %}
        {%- set last_user_index = i %}
    {%- endif %}
{%- endfor %}

{%- for i in range(messages | length) %}
    {%- set message = messages[i] %}
    {%- if message['role'] == 'system' %}
<|im_start|>system
{{ message['content'] }}<|im_end|>
    {%- elif message['role'] == 'context' %}
<|im_start|>context
{{ message['content'] }}<|im_end|>
    {%- elif message['role'] == 'user' %}
<|im_start|>user
{{ message['content'] }}<|im_end|>
    {%- elif message['role'] == 'assistant' %}
<|im_start|>assistant
    {%- if i > last_user_index and (i + 1 == messages | length or messages[i+1]['role'] != 'assistant') %}
<think>

</think>
    {%- endif %}
{{ message['content'] }}<|im_end|>
    {%- endif %}
{%- endfor %}

{%- if add_generation_prompt %}
<|im_start|>assistant
    {%- if enable_thinking is defined and enable_thinking == false %}
<think>

</think>
    {%- endif %}
{%- endif %}"""
# 3. Asignar la plantilla personalizada
tokenizer.chat_template = custom_chat_template

# 4. Verificar que funciona
messages = [
    {"role": "system", "content": "Responde usando el contexto"},
    {"role": "context", "content": "Qwen 2.5 tiene 128K de contexto"},
    {"role": "user", "content": "¿Cuánto contexto soporta Qwen?"},
]

formatted = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking = False
)

print("Formatted output:\n", formatted)


messages = [
    {"role": "system", "content": "Responde usando el contexto"},
    {"role": "context", "content": "Qwen 2.5 tiene 128K de contexto"},
    {"role": "user", "content": "¿Cuánto contexto soporta Qwen?"},
    {"role": "assistant", "content": "Soporta todo"}
]

formatted = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=False
)

print("Formatted output:\n", formatted)

tensor([ 0.0061, -0.0928,  0.0251,  ...,  0.0051, -0.0082, -0.0054],
       device='cuda:0', dtype=torch.bfloat16)
Formatted output:
 <|im_start|>system
Responde usando el contexto<|im_end|><|im_start|>context
Qwen 2.5 tiene 128K de contexto<|im_end|><|im_start|>user
¿Cuánto contexto soporta Qwen?<|im_end|><|im_start|>assistant<think>

</think>
Formatted output:
 <|im_start|>system
Responde usando el contexto<|im_end|><|im_start|>context
Qwen 2.5 tiene 128K de contexto<|im_end|><|im_start|>user
¿Cuánto contexto soporta Qwen?<|im_end|><|im_start|>assistant<think>

</think>Soporta todo<|im_end|>


In [21]:
token_id = tokenizer.convert_tokens_to_ids("<|im_start|>context")
print(model.get_input_embeddings().weight[token_id])

tensor([ 0.0061, -0.0928,  0.0251,  ...,  0.0051, -0.0082, -0.0054],
       device='cuda:0', dtype=torch.bfloat16)


In [4]:
from utils import read_fragment_doc
general_contact_information = read_fragment_doc("./documentos/informacion_general_contacto.txt")
general_contact_information

'Escuelas Profesionales de la Facultad de Ciencias de la UNI\n\nInformación de Contacto:\nSegún la escuela profesional (carrera universitaria), el correo de contacto es el siguiente:\n\nPara Física, Química o Ciencia de la Computación: escuelas_fc1@uni.edu.pe\nPara Matemáticas o Ingeniería Física: escuelas_fc2@uni.edu.pe\n\nHorario de Atención de Escuelas Profesionales: De Lunes a Viernes de 8:30 a.m a 4:00 p.m\n\nÁrea de Estadística y Registros Académicos de la Facultad de Ciencias de la UNI\n\nInformación de Contacto de la oficina de estadística (AERA)\nE-mail: estadistica_fc@uni.edu.pe\n\nHorario de Atención de la oficina de estadística (AERA):\nDe Lunes a Viernes de 8:00 a.m. a 4:00 p.m.'

In [5]:
print(tokenizer.chat_template)

{%- set last_user_index = -1 %}
{%- for i in range(messages | length) %}
    {%- if messages[i]['role'] == 'user' %}
        {%- set last_user_index = i %}
    {%- endif %}
{%- endfor %}

{%- for i in range(messages | length) %}
    {%- set message = messages[i] %}
    {%- if message['role'] == 'system' %}
<|im_start|>system
{{ message['content'] }}<|im_end|>
    {%- elif message['role'] == 'context' %}
<|im_start|>context
{{ message['content'] }}<|im_end|>
    {%- elif message['role'] == 'user' %}
<|im_start|>user
{{ message['content'] }}<|im_end|>
    {%- elif message['role'] == 'assistant' %}
<|im_start|>assistant
    {%- if i > last_user_index and (i + 1 == messages | length or messages[i+1]['role'] != 'assistant') %}
<think>

</think>
    {%- endif %}
{{ message['content'] }}<|im_end|>
    {%- endif %}
{%- endfor %}

{%- if add_generation_prompt %}
<|im_start|>assistant
    {%- if enable_thinking is defined and enable_thinking == false %}
<think>

</think>
    {%- endif %}
{%- end

In [6]:
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

prompt_role_system = """Eres Aerito, un asistente de IA especializado en temas de matrícula, procedimientos y trámites académicos de la Facultad de Ciencias de la Universidad Nacional de Ingeniería del Perú.
Deberás responder a los mensajes asegurándote de cumplir con los siguientes criterios.
    1. Proporcionar respuestas informativas, útiles y concisas a las preguntas del usuario basándote exclusivamente en el contexto proporcionado.
    2. Mantén un tono cordial, empático y servicial en sus interacciones.
    3. Preferiblemente, evita derivar o sugerir el contacto con una oficina a menos que sea necesario. Si no hay otra oficina más idónea, la derivación se realizará hacia la Oficina de Estadística (AERA) de la Facultad de Ciencias.
    4. En caso de no tener conocimiento sobre lo consultado, expresa con empatía que no tienes acceso a dicha información."""

#tokenizer = get_chat_template(
#    tokenizer,
#    chat_template = "qwen-2.5"
#)

def formatting_prompts_func(examples):
    convos = examples["turns"]
    recovered_texts = examples["retrieved_texts"]
    docs_range = examples["docs_range"]

    #"recovered_texts= examples["recovered_texts"],
    for convo, texts, doc_range in zip(convos, recovered_texts, docs_range):
        if convo[0]["role"] != "system":
            convo.insert(0, {
                "role": "system",
                "content": prompt_role_system
            })
            
            if texts is not None:
                #print("texts", texts)
                texts_context = [text["text"] for text in texts[doc_range[0]:doc_range[1]]] + [general_contact_information]
                context = "\n\n".join(texts_context)
            
                convo.insert(1, {
                    "role": "context",
                    "content": context
                })
                #print("convo[1]",convo[1])
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

dataset = load_dataset("json", data_files = {"train":"./conversational_faq/data/train_turns_dataset.json", "val": "./conversational_faq/data/val_turns_dataset.json"})
dataset = dataset.map(formatting_prompts_func, batched=True)
print(dataset['train'][0]['text'])

<|im_start|>system
Eres Aerito, un asistente de IA especializado en temas de matrícula, procedimientos y trámites académicos de la Facultad de Ciencias de la Universidad Nacional de Ingeniería del Perú.
Deberás responder a los mensajes asegurándote de cumplir con los siguientes criterios.
    1. Proporcionar respuestas informativas, útiles y concisas a las preguntas del usuario basándote exclusivamente en el contexto proporcionado.
    2. Mantén un tono cordial, empático y servicial en sus interacciones.
    3. Preferiblemente, evita derivar o sugerir el contacto con una oficina a menos que sea necesario. Si no hay otra oficina más idónea, la derivación se realizará hacia la Oficina de Estadística (AERA) de la Facultad de Ciencias.
    4. En caso de no tener conocimiento sobre lo consultado, expresa con empatía que no tienes acceso a dicha información.<|im_end|><|im_start|>context
Fecha De Pago De Autoseguro
¿Hasta que fecha se debe realizar el pago por concepto de autoseguro estudiant

In [7]:
from unsloth import is_bf16_supported
from trl import SFTConfig, SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    eval_dataset = dataset['val'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        dataset_text_field= "text",
        per_device_train_batch_size = 16,
        gradient_accumulation_steps = 4,
        per_device_eval_batch_size = 8,
        warmup_steps=5,
        num_train_epochs=2,
        #max_steps=60,
        learning_rate= 2e-4,
        fp16= not is_bf16_supported(),
        bf16= is_bf16_supported(),
        logging_steps=1,
        eval_strategy= "epoch",
        save_strategy= "epoch",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        packing = False
    ),
)

In [9]:
print(tokenizer.decode(trainer.train_dataset[1]["input_ids"]))

<|im_start|>system
Eres Aerito, un asistente de IA especializado en temas de matrícula, procedimientos y trámites académicos de la Facultad de Ciencias de la Universidad Nacional de Ingeniería del Perú.
Deberás responder a los mensajes asegurándote de cumplir con los siguientes criterios.
    1. Proporcionar respuestas informativas, útiles y concisas a las preguntas del usuario basándote exclusivamente en el contexto proporcionado.
    2. Mantén un tono cordial, empático y servicial en sus interacciones.
    3. Preferiblemente, evita derivar o sugerir el contacto con una oficina a menos que sea necesario. Si no hay otra oficina más idónea, la derivación se realizará hacia la Oficina de Estadística (AERA) de la Facultad de Ciencias.
    4. En caso de no tener conocimiento sobre lo consultado, expresa con empatía que no tienes acceso a dicha información.<|im_end|><|im_start|>context
Generación De Orden De Pago Para Autoseguro
¿Cual es el proceso para que un ingresante o alumno regular ge

In [10]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA RTX 4500 Ada Generation. Max memory = 23.994 GB.
1.002 GB of memory reserved.


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,821 | Num Epochs = 2 | Total steps = 182
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 4 x 1) = 64
 "-____-"     Trainable parameters = 40,370,176/600,000,000 (6.73% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Epoch,Training Loss,Validation Loss
1,0.179100,0.168058
2,0.124800,0.136976


Unsloth: Not an error, but Qwen3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient
/home/anthony/miniconda3/envs/llm/lib/python3.11/site-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/home/anthony/miniconda3/envs/llm/lib/python3.11/site-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [12]:
token_id = tokenizer.convert_tokens_to_ids("<|im_start|>context")
print(model.get_input_embeddings().weight[token_id])

tensor([ 0.0061, -0.0928,  0.0251,  ...,  0.0051, -0.0082, -0.0054],
       device='cuda:0', dtype=torch.bfloat16)


In [13]:
# 🔁 FUSIONAR EL MODELO
model = model.merge_and_unload()  # Esto remueve LoRA y fusiona los pesos

# ✅ GUARDAR EL MODELO Y TOKENIZER
model.save_pretrained("models/qwen3-finetuned-merge_and_unload")
tokenizer.save_pretrained("models/qwen3-finetuned-merge_and_unload")

/home/anthony/miniconda3/envs/llm/lib/python3.11/site-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('models/qwen3-finetuned-merge_and_unload/tokenizer_config.json',
 'models/qwen3-finetuned-merge_and_unload/special_tokens_map.json',
 'models/qwen3-finetuned-merge_and_unload/chat_template.jinja',
 'models/qwen3-finetuned-merge_and_unload/vocab.json',
 'models/qwen3-finetuned-merge_and_unload/merges.txt',
 'models/qwen3-finetuned-merge_and_unload/added_tokens.json',
 'models/qwen3-finetuned-merge_and_unload/tokenizer.json')

In [14]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name =  "./models/qwen3-finetuned-merge_and_unload",
    max_seq_length = max_seq_length,
    load_in_4bit = True)

==((====))==  Unsloth 2025.6.2: Fast Qwen3 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA RTX 4500 Ada Generation. Num GPUs = 1. Max memory: 23.994 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [15]:
print(tokenizer.chat_template)

{%- set last_user_index = -1 %}
{%- for i in range(messages | length) %}
    {%- if messages[i]['role'] == 'user' %}
        {%- set last_user_index = i %}
    {%- endif %}
{%- endfor %}

{%- for i in range(messages | length) %}
    {%- set message = messages[i] %}
    {%- if message['role'] == 'system' %}
<|im_start|>system
{{ message['content'] }}<|im_end|>
    {%- elif message['role'] == 'context' %}
<|im_start|>context
{{ message['content'] }}<|im_end|>
    {%- elif message['role'] == 'user' %}
<|im_start|>user
{{ message['content'] }}<|im_end|>
    {%- elif message['role'] == 'assistant' %}
<|im_start|>assistant
    {%- if i > last_user_index and (i + 1 == messages | length or messages[i+1]['role'] != 'assistant') %}
<think>

</think>
    {%- endif %}
{{ message['content'] }}<|im_end|>
    {%- endif %}
{%- endfor %}

{%- if add_generation_prompt %}
<|im_start|>assistant
    {%- if enable_thinking is defined and enable_thinking == false %}
<think>

</think>
    {%- endif %}
{%- end

In [13]:
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer

dtype = None

FastLanguageModel.for_inference(model)


messages = dataset['train'][0]['turns'][:-1]

streamer = TextStreamer(tokenizer)

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    enable_thinking = False,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    input_ids = inputs , 
    max_new_tokens = 500, 
    use_cache = True, 
    streamer = streamer
    )


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|im_start|>system
Eres Aerito, un asistente de IA especializado en temas de matrícula, procedimientos y trámites académicos de la Facultad de Ciencias de la Universidad Nacional de Ingeniería del Perú.
Deberás responder a los mensajes asegurándote de cumplir con los siguientes criterios.
    1. Proporcionar respuestas informativas, útiles y concisas a las preguntas del usuario basándote exclusivamente en el contexto proporcionado.
    2. Mantén un tono cordial, empático y servicial en sus interacciones.
    3. Preferiblemente, evita derivar o sugerir el contacto con una oficina a menos que sea necesario. Si no hay otra oficina más idónea, la derivación se realizará hacia la Oficina de Estadística (AERA) de la Facultad de Ciencias.
    4. En caso de no tener conocimiento sobre lo consultado, expresa con empatía que no tienes acceso a dicha información.<|im_end|><|im_start|>context
Fecha De Pago De Autoseguro
¿Hasta que fecha se debe realizar el pago por concepto de autoseguro estudiant

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

#model = AutoModelForCausalLM.from_pretrained("models/qwen3-finetuned-merged", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("models/qwen3-finetuned-merged")
print(tokenizer.get_vocab().get("<|im_start|>context"))  # Debería imprimir un ID, no None
print(tokenizer.get_vocab().get("<|context|>"))
model.save_pretrained("models/qwen3-finetuned")
tokenizer.save_pretrained("models/qwen3-finetuned")
model.save_pretrained_merged("models/qwen3-finetuned-16bit", tokenizer, save_method = "merged_16bit",)
model.save_pretrained_merged("models/qwen3-finetuned-4bit", tokenizer, save_method = "merged_4bit_forced",)
model.save_pretrained_merged("models/qwen3-finetuned-merged", tokenizer)
model.save_pretrained_merged("models/qwen3-finetuned-merged", tokenizer)

In [19]:
messages

[{'content': 'Eres Aerito, un asistente de IA especializado en temas de matrícula, procedimientos y trámites académicos de la Facultad de Ciencias de la Universidad Nacional de Ingeniería del Perú.\nDeberás responder a los mensajes asegurándote de cumplir con los siguientes criterios.\n    1. Proporcionar respuestas informativas, útiles y concisas a las preguntas del usuario basándote exclusivamente en el contexto proporcionado.\n    2. Mantén un tono cordial, empático y servicial en sus interacciones.\n    3. Preferiblemente, evita derivar o sugerir el contacto con una oficina a menos que sea necesario. Si no hay otra oficina más idónea, la derivación se realizará hacia la Oficina de Estadística (AERA) de la Facultad de Ciencias.\n    4. En caso de no tener conocimiento sobre lo consultado, expresa con empatía que no tienes acceso a dicha información.',
  'role': 'system'},
 {'content': 'Fecha De Pago De Autoseguro\n¿Hasta que fecha se debe realizar el pago por concepto de autoseguro 

In [17]:
from unsloth.chat_templates import get_chat_template
from transformers import TextStreamer

#tokenizer = get_chat_template(
#    tokenizer,
#    chat_template = "qwen-2.5",
#)


FastLanguageModel.for_inference(model)


messages = dataset['train'][5]['turns'][:-1]

streamer = TextStreamer(tokenizer)

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    enable_thinking = False,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    input_ids = inputs , 
    max_new_tokens = 256, 
    use_cache = True, 
    streamer = streamer,
    temperature = 1.5, 
    min_p = 0.1)


#print(tokenizer.batch_decode(outputs)[0])

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|im_start|>system
Eres Aerito, un asistente de IA especializado en temas de matrícula, procedimientos y trámites académicos de la Facultad de Ciencias de la Universidad Nacional de Ingeniería del Perú.
Deberás responder a los mensajes asegurándote de cumplir con los siguientes criterios.
    1. Proporcionar respuestas informativas, útiles y concisas a las preguntas del usuario basándote exclusivamente en el contexto proporcionado.
    2. Mantén un tono cordial, empático y servicial en sus interacciones.
    3. Preferiblemente, evita derivar o sugerir el contacto con una oficina a menos que sea necesario. Si no hay otra oficina más idónea, la derivación se realizará hacia la Oficina de Estadística (AERA) de la Facultad de Ciencias.
    4. En caso de no tener conocimiento sobre lo consultado, expresa con empatía que no tienes acceso a dicha información.<|im_end|>
<|im_start|>context
Fecha De Pago De Autoseguro
¿Hasta que fecha se debe realizar el pago por concepto de autoseguro estudian

In [14]:
from tqdm import tqdm

torch.manual_seed(42)

FastLanguageModel.for_inference(model)

generation_outputs = dict(messages = [], pred_outputs = [], true_outputs = [])

val_dataset = dataset['val']

# Preparamos las entradas (messages + labels separados)
inputs_list = []
true_outputs = []
messages_list = []

tokenizer.padding_side = "left"  # importante para generación

for conv in tqdm(val_dataset,desc = "Preparate Conversations"):
    messages = conv['turns'][:-1]
    messages_list.append(messages)
    input = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    inputs_list.append(input)
    true_outputs.append(conv['turns'][-1]["content"])

# Procesamos por batch
batch_size = 12

for i in tqdm(range(0, len(inputs_list), batch_size), desc="Batching"):
    batch_inputs = inputs_list[i:i+batch_size]
    batch_truths = true_outputs[i:i+batch_size]
    batch_messages = messages_list[i:i+batch_size]

    # Tokenizar y padear batch
    tokenized = tokenizer(
        batch_inputs,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to("cuda")

    with torch.inference_mode():
        outputs = model.generate(
            input_ids=tokenized["input_ids"],
            attention_mask=tokenized["attention_mask"],
            max_new_tokens=256,
            use_cache=True,
        )
        

    # Decodificar solo lo nuevo generado por cada ejemplo
    for j in range(len(outputs)):
        input_len = tokenized["input_ids"][j].shape[0]
        generated_tokens = outputs[j][input_len:]
        generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

        generation_outputs["messages"].append(batch_messages[j])
        generation_outputs["pred_outputs"].append(generated_text)
        generation_outputs["true_outputs"].append(batch_truths[j])

Batching:   0%|          | 0/160 [00:00<?, ?it/s]/home/anthony/miniconda3/envs/llm/lib/python3.11/site-packages/unsloth/kernels/utils.py:443: UserWarning: An output with one or more elements was resized since it had shape [1, 12, 1024], which does not match the required output shape [12, 1, 1024]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at /pytorch/aten/src/ATen/native/Resize.cpp:30.)
  out = torch_matmul(X, W, out = out)
Batching:  99%|█████████▉| 159/160 [2:33:13<00:57, 57.32s/it]  /home/anthony/miniconda3/envs/llm/lib/python3.11/site-packages/unsloth/kernels/utils.py:443: UserWarning: An output with one or more elements was resized since it had shape [1, 11, 1024], which does not match the required output shape [11, 1, 1024]. This behavior is deprecated, and in a future PyTorch rel

In [16]:
import pandas as pd
df = pd.DataFrame(generation_outputs)
df.to_csv("resultados.csv")

In [17]:
df

,messages,pred_outputs,true_outputs
0,"[{'content': 'Eres Aerito, un asistente de IA ...",<think>\n\n</think>El manual de pagos para obt...,El Manual de Pagos está disponible en la secci...
1,"[{'content': 'Eres Aerito, un asistente de IA ...","<think>\n\n</think>Lamentablemente, no tengo i...",No hay un plazo específico mencionado para gen...
2,"[{'content': 'Eres Aerito, un asistente de IA ...","<think>\n\n</think>Sí, puedes consultar el est...","Sí, puedes consultar el estado de tu matrícula..."
3,"[{'content': 'Eres Aerito, un asistente de IA ...","<think>\n\n</think>Lamentablemente, no tengo a...",Los problemas comunes que los estudiantes suel...
4,"[{'content': 'Eres Aerito, un asistente de IA ...","<think>\n\n</think>Sí, puedes contactar a la O...",La Oficina de Estadística (AERA) de la Faculta...
...,...,...,...
1914,"[{'content': 'Eres Aerito, un asistente de IA ...",\n</think>Para completar el trámite del correo...,"Lamentablemente, no tengo información sobre ot..."
1915,"[{'content': 'Eres Aerito, un asistente de IA ...","el, para más información.\n\nEscuelas Profesi...",¡De nada! Me alegra haber podido ayudarte. Si ...
1916,"[{'content': 'Eres Aerito, un asistente de IA ...",de la UNI. \n\nEscuelas Profesionales de la F...,Debes adjuntar documentos que acrediten la raz...
1917,"[{'content': 'Eres Aerito, un asistente de IA ...",el ciclo académico.\n\nEscuelas Profesionales...,"Sí, hay un formato específico que debes seguir..."


In [ ]:
from tqdm import tqdm

torch.manual_seed(42)

FastLanguageModel.for_inference(model)
generation_outputs = dict(messages = [], pred_outputs = [], true_outputs = [])

val_dataset = dataset['val']

for conv in tqdm(val_dataset, desc= "Conversations"):
    messages = conv['turns'][:-1]
    
    generation_outputs['messages'].append(messages)

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True,
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(
        input_ids = inputs , 
        max_new_tokens = 256, 
        use_cache = False, 
        temperature = 0.0,
        do_sample = False)
    
    text_output = tokenizer.decode(outputs[0, len(inputs[0]):], skip_special_tokens = True)
    generation_outputs['pred_outputs'].append(text_output)
    generation_outputs['true_outputs'].append(conv['turns'][-1]["content"])
    

Conversations:   0%|          | 0/1919 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [43]:
import pandas as pd
pd.DataFrame(generation_outputs)

,messages,pred_outputs,true_outputs
0,"[{'content': 'Eres Aerito, un asistente de IA ...",El Manual de Pagos está disponible en la secci...,El Manual de Pagos está disponible en la secci...


In [ ]:
import json
import os
from utils import save_json
from glob import glob
from tqdm import tqdm

convs_sim_file = "./conversational_faq/openline-questions/conv_sim_derived_questions_faq_33_0_to_7.json"

files = glob("./conversational_faq/openline-questions/*.json")

for file_path in tqdm(files, desc= "File"): 
    with open(file_path, "r") as f:
        convs_data = json.load(f)

    save_dir = "./conversational_faq/openline-questions-fixed"

    convs_data_fix = convs_data.copy()

    for turns_data in convs_data_fix:
        for turn_data in turns_data["messages"]:
            if turn_data['role'] == "user":
                context = turn_data['context']
                recovered_texts = turn_data['recovered_texts']
                max_idx = 0
                
                if recovered_texts is None:
                    continue

                for i, rt in enumerate(recovered_texts):
                    if rt['text'] in context:
                        max_idx = i
                        #print(f"{i}: encontrado")
                    #else:
                        #print(f"{i}: no encontrado")

                docs_range = [0, max_idx]
                turn_data['docs_range'] = docs_range

    base_name = os.path.basename(file_path)[:-5]
    save_json(save_dir, base_name, convs_data_fix)

In [ ]:
context =  "\n\nRequisitos Y Procedimiento Para Matricula Solo Para Ingresantes\n\u00bfCuales son los requisitos y el proceso de matr\u00edcula para nuevos ingresantes de pregrado en la Facultad de Ciencias de la Universidad Nacional de Ingenier\u00eda?\nLos nuevos estudiantes de pregrado ingresantes a la Facultad de Ciencias deber\u00e1n seguir el siguiente proceso, por \u00fanica vez, para completar su primera matricula (primera matricula al primer ciclo):\n    1. Recabar su constancia de ingreso. Luego de gestionar la emisi\u00f3n de su constancia de ingreso, la Direcci\u00f3n de Admisi\u00f3n (DIAD) de la UNI enviar\u00e1 a su correo la constancia de ingreso.\n    2. Actualizaci\u00f3n de datos en intranet. DIRCE har\u00e1 llegar a su correo su clave para acceder a la plataforma de intranet-alumnos y completar sus datos.\n    3. Registrar los datos en la Facultad de Ciencias. La oficina de estad\u00edstica de la facultad enviar\u00e1 al correo del ingresante la ficha de datos. El llenado es car\u00e1cter obligatorio.\n    4. Efectuar el pago por autoseguro (S/. 55.00) en el plazo establecido en el cronograma. Para ello, primero generar una orden de pago a traves de la plataforma intranet-alumnos.\n    5. Realizar la entrega de los siguientes documentos a la oficina de estad\u00edstica seg\u00fan el cronograma de actividades de matr\u00edcula para ingresantes publicado dentro de la secci\u00f3n 'MATR\u00cdCULA Y PROCEDIMIENTOS' en la p\u00e1gina web de la Facultad de Ciencias.\n        - Constancia de Ingreso\n        - Ficha de datos\n        - Constancia de Evaluaci\u00f3n Socioecon\u00f3mica\n        - Certificado M\u00e9dico expedido por el Centro Medico UNI\n        - Comprobante de pago de Autoseguro Estudiantil\n    6. La oficina de estad\u00edstica ejecutar\u00e1 la matr\u00edcula (inscripci\u00f3n de cursos) de los ingresantes seg\u00fan el cronograma de actividades, \u00fanicamente para aquellos que hayan cumplido con la entrega de la entrega de los documentos requeridos en de las fechas establecidas en el cronograma.\n    7. Los ingresantes matriculados recibir\u00e1n un correo con el horario de sus cursos y tambi\u00e9n podr\u00e1n visualizar sus cursos y horarios en la plataforma intranet-alumnos.\n\nRequisitos Y Procedimiento De Matricula Para Estudiantes Regulares\n\u00bfCuales son los requisitos y el proceso de matr\u00edcula regular al ciclo acad\u00e9mico para alumnos regulares de pregrado en la Facultad de Ciencias de la Universidad Nacional de Ingenier\u00eda?\nLos estudiantes regulares de pregrado en la Facultad de Ciencias de la UNI deben cumplir con siguientes requisitos y procedimiento para realizar su matr\u00edcula al ciclo acad\u00e9mico.\n\nRequisitos para la matricula:\n1. Haber efectuado el pago del autoseguro estudiantil mediante la plataforma intranet-alumnos.\n\n2. No tener adeudos pendientes con la Facultad, ya sea por concepto econ\u00f3mico, pr\u00e9stamos de libros u otros materiales de ense\u00f1anza en general de la biblioteca.\n\n3. En caso de estar en riesgo acad\u00e9mico, haber completado la tutor\u00eda obligatoria previa a la matr\u00edcula.\n\n4. Si el estudiante ha dejado de estudiar, reservado matr\u00edcula o solicitado retiro total en el \u00faltimo per\u00edodo, deber\u00e1 tramitar previamente su reincorporaci\u00f3n a trav\u00e9s de la plataforma intranet-alumnos.\n\nProcedimiento para la matr\u00edcula regular:\n1. Generar la orden de pago del autoseguro estudiantil a trav\u00e9s de la plataforma INTRALU. Para ello, haz clic en \"Tr\u00e1mites\" para acceder al m\u00f3dulo de tr\u00e1mites y pagos. Luego, selecciona el tr\u00e1mite correspondiente, haz clic en \"Nuevo\" y sigue las instrucciones para completar el proceso.\n2. Realizar el pago del autoseguro dentro del plazo establecido en el calendario de actividades acad\u00e9micas, acerc\u00e1ndose a un sucursal del BCP o a traves de la aplicaci\u00f3n movil del banco. No se requiere la entrega del voucher de pago del autoseguro ni otros documentos.\n3. En la fecha y turno asignado, el estudiante llevar\u00e1 a cabo a su matr\u00edcula de forma virtual, es decir, realizar\u00e1 la inscripci\u00f3n de sus cursos a trav\u00e9s de la plataforma intranet-alumnos. Acceder\u00e1 al m\u00f3dulo de Matr\u00edcula UNI dentro de esta plataforma, seleccionar\u00e1 las asignaturas y horarios deseados, y finalizar\u00e1 confirmando la matr\u00edcula en todas las asignaturas elegidas en el sistema.\n\nLos turnos de matr\u00edcula se asignan por grupos seg\u00fan el promedio ponderado de los dos \u00faltimos ciclos acad\u00e9micos, en orden de m\u00e9rito. Los estudiantes pueden consultarlos en el m\u00f3dulo de Matr\u00edcula UNI de la intranet-alumnos.\n\nMatricula Ingresantes\nLos nuevos ingresantes a la Facultad de Ciencias de la Universidad Nacional de Ingenieria, deberan seguir realizar el siguiente proceso para completar su matricula (primera matricula al primer ciclo):\n    1. Recabar su constancia de ingreso. Luego de gestionar la emisi\u00f3n de su constancia de ingreso, la Direcci\u00f3n de Admisi\u00f3n (DIAD) de la UNI enviar\u00e1 a su correo electr\u00f3nico la constancia de ingreso. Para m\u00e1s detalles sobre el proceso, puede consultar con DIAD al correo informes.admision@uni.edu.pe. \n    2. Actualizacion de datos en DIRCE. La Direcci\u00f3n de Registros Central y Estad\u00edstica har\u00e1 llegar a su correo su clave para acceder a la plataforma para intranet-alumnos y completar sus datos.\n    3. Registrar los datos en la Facultad de Ciencias. La oficina de estad\u00edstica enviar\u00e1 al correo del ingresante la ficha de datos. El llenado es car\u00e1cter obligatorio.\n    4. Efectuar el pago por autoseguro dentro del plazo establecido en el cronograma de actividades academicas publicado en la pagina web de la Facultad de ciencias. Para ello, el ingresante debera generar una orden de pago atravez del portal INTRALU, luego podra realizar el pago por aplicativo movil BCP o en alguna sucursal de la entidad bancaria.\n    5. Realizar la entrega de los siguientes documentos a la oficina de estad\u00edstica seg\u00fan el cronograma de actividades de matr\u00edcula de ingresantes publicado en la secci\u00f3n 'MATR\u00cdCULA Y PROCEDIMIENTOS' en la p\u00e1gina web de la Facultad de Ciencias.\n        - Contancia de Ingreso\n        - Ficha de datos Personales. Enviada por la oficina de estad\u00edstica al correo del ingresante.\n        - Constancia de Evaluacion Socioeconomica. El ingresante debera acudir a la cita para su evaluacion socioeconomica que le llegara a su correo. \n        - Certificado M\u00e9dico expedido por el Centro Medico UNI. Para ello acudir a su examen medico segun los cronograma publicados por la Direcci\u00f3n de Admisi\u00f3n. \n        - Comprobante de pago de Autoseguro Estudiantil\n    6. AERA ejecutar\u00e1 la matr\u00edcula (inscripci\u00f3n de cursos) de los ingresantes seg\u00fan el cronograma de actividades, \u00fanicamente para aquellos que hayan cumplido con la entrega de la entrega de los documentos requeridos en de las fechas establecidas en el cronograma.\n    7. Los ingresantes matriculados recibir\u00e1n un mensaje en su correo con el horario de sus cursos, ademas podran visualizar los cursos y horarios dentro del portal INTRALU.\n\nPara generar una orden de pago, el ingresante debe ingresar a la plataforma INTRALU. En esta plataforma, selecciona la opci\u00f3n \"Tr\u00e1mites\" para acceder al m\u00f3dulo de pagos. En dicho m\u00f3dulo, selecciona el tr\u00e1mite correspondiente en el men\u00fa desplegable, haz clic en el bot\u00f3n \"Nuevo\" y sigue las indicaciones hasta generar la orden de pago. Para mas detalles sobre el proceso para generar un orden de pogo puede revisar el manual de pagos de la UNI publicado en la pagina web de la Facultad de Ciencias.\n\n\nTraslado Interno Procedimiento Y Requisitos\n\u00bfC\u00f3mo es el procedimiento para que un estudiante de pregrado de la Facultad de Ciencias realice un traslado interno a otra carrera dentro de la misma Facultad?\nPara llevar a cabo un traslado interno, un estudiante de la Facultad de Ciencias deber\u00e1 seguir el siguiente procedimiento:\n1. Realizar el pago correspondiente por concepto de primer o segundo traslado interno, seg\u00fan sea el caso (S/ 125.20 o S/ 300.00). Para ello, el estudiante deber\u00e1 generar una orden de pago a trav\u00e9s de la plataforma INTRALU y efectuar el pago en una agencia del BCP o mediante el aplicativo m\u00f3vil del banco.\n2. El estudiante deber\u00e1 presentar su solicitud a trav\u00e9s de el plataforma intranet-alumnos, eligiendo la opci\u00f3n \"Traslado Interno\" en el men\u00fa \"Mis Tr\u00e1mites\". Esta solicitud deber\u00e1 ser enviada a traves de la plataforma en la fecha indicada en el calendario de actividades acad\u00e9micas, adjuntando el comprobante de pago por concepto de traslado interno y la ficha acad\u00e9mica.\n3. La Escuela verificar\u00e1 los documentos y requisitos propios dentro del primer d\u00eda \u00fatil de presentada la solicitud, registrar\u00e1 el expediente y remitir\u00e1 un correo de recepci\u00f3n al interesado.\n4. El director de Escuela o la Comisi\u00f3n de Matr\u00edcula analizar\u00e1n las solicitudes y, utilizando el formato disponible en la web DIRCE, inscribir\u00e1n las convalidaciones pertinentes seg\u00fan el Avance Curricular del alumno, las notas y las normas de Convalidaci\u00f3n.\n5. Con la autorizaci\u00f3n del director, el expediente ser\u00e1 enviado al Decano y posteriormente presentado ante el Consejo de Facultad para su aprobaci\u00f3n mediante Resoluci\u00f3n Decanal y remisi\u00f3n a la DIRCE.\n6. La DIRCE ejecutar\u00e1 el cambio de especialidad y realizar\u00e1 las convalidaciones.\n7. La Oficina de Estad\u00edstica gestionar\u00e1 la matr\u00edcula de los estudiantes por traslado interno en funci\u00f3n a la solicitud de matr\u00edcula del director de la Escuela Profesional, y finalmente, la DIRCE realizar\u00e1 la ejecuci\u00f3n de la matr\u00edcula en el sistema SIGA.\n8. No se recibir\u00e1n solicitudes extempor\u00e1neas.\nEscuelas Profesionales de la Facultad de Ciencias de la UNI\n\nInformaci\u00f3n de Contacto:\nSeg\u00fan la escuela profesional (carrera universitaria), el correo de contacto es el siguiente:\n\nPara F\u00edsica, Qu\u00edmica o Ciencia de la Computaci\u00f3n: escuelas_fc1@uni.edu.pe\nPara Matem\u00e1ticas o Ingenier\u00eda F\u00edsica: escuelas_fc2@uni.edu.pe\n\nHorario de Atenci\u00f3n de Escuelas Profesionales: De Lunes a Viernes de 8:30 a.m a 4:00 p.m\n\n\u00c1rea de Estad\u00edstica y Registros Acad\u00e9micos de la Facultad de Ciencias de la UNI\n\nInformaci\u00f3n de Contacto de la oficina de estad\u00edstica (AERA)\nE-mail: estadistica_fc@uni.edu.pe\n\nHorario de Atenci\u00f3n de la oficina de estad\u00edstica (AERA):\nDe Lunes a Viernes de 8:00 a.m. a 4:00 p.m."
recovered_texts = [
                    {
                        "text": "Requisitos Y Procedimiento Para Matricula Solo Para Ingresantes\n\u00bfCuales son los requisitos y el proceso de matr\u00edcula para nuevos ingresantes de pregrado en la Facultad de Ciencias de la Universidad Nacional de Ingenier\u00eda?\nLos nuevos estudiantes de pregrado ingresantes a la Facultad de Ciencias deber\u00e1n seguir el siguiente proceso, por \u00fanica vez, para completar su primera matricula (primera matricula al primer ciclo):\n    1. Recabar su constancia de ingreso. Luego de gestionar la emisi\u00f3n de su constancia de ingreso, la Direcci\u00f3n de Admisi\u00f3n (DIAD) de la UNI enviar\u00e1 a su correo la constancia de ingreso.\n    2. Actualizaci\u00f3n de datos en intranet. DIRCE har\u00e1 llegar a su correo su clave para acceder a la plataforma de intranet-alumnos y completar sus datos.\n    3. Registrar los datos en la Facultad de Ciencias. La oficina de estad\u00edstica de la facultad enviar\u00e1 al correo del ingresante la ficha de datos. El llenado es car\u00e1cter obligatorio.\n    4. Efectuar el pago por autoseguro (S/. 55.00) en el plazo establecido en el cronograma. Para ello, primero generar una orden de pago a traves de la plataforma intranet-alumnos.\n    5. Realizar la entrega de los siguientes documentos a la oficina de estad\u00edstica seg\u00fan el cronograma de actividades de matr\u00edcula para ingresantes publicado dentro de la secci\u00f3n 'MATR\u00cdCULA Y PROCEDIMIENTOS' en la p\u00e1gina web de la Facultad de Ciencias.\n        - Constancia de Ingreso\n        - Ficha de datos\n        - Constancia de Evaluaci\u00f3n Socioecon\u00f3mica\n        - Certificado M\u00e9dico expedido por el Centro Medico UNI\n        - Comprobante de pago de Autoseguro Estudiantil\n    6. La oficina de estad\u00edstica ejecutar\u00e1 la matr\u00edcula (inscripci\u00f3n de cursos) de los ingresantes seg\u00fan el cronograma de actividades, \u00fanicamente para aquellos que hayan cumplido con la entrega de la entrega de los documentos requeridos en de las fechas establecidas en el cronograma.\n    7. Los ingresantes matriculados recibir\u00e1n un correo con el horario de sus cursos y tambi\u00e9n podr\u00e1n visualizar sus cursos y horarios en la plataforma intranet-alumnos.",
                        "relatedness": 0.7661936283111572
                    },
                    {
                        "text": "Requisitos Y Procedimiento De Matricula Para Estudiantes Regulares\n\u00bfCuales son los requisitos y el proceso de matr\u00edcula regular al ciclo acad\u00e9mico para alumnos regulares de pregrado en la Facultad de Ciencias de la Universidad Nacional de Ingenier\u00eda?\nLos estudiantes regulares de pregrado en la Facultad de Ciencias de la UNI deben cumplir con siguientes requisitos y procedimiento para realizar su matr\u00edcula al ciclo acad\u00e9mico.\n\nRequisitos para la matricula:\n1. Haber efectuado el pago del autoseguro estudiantil mediante la plataforma intranet-alumnos.\n\n2. No tener adeudos pendientes con la Facultad, ya sea por concepto econ\u00f3mico, pr\u00e9stamos de libros u otros materiales de ense\u00f1anza en general de la biblioteca.\n\n3. En caso de estar en riesgo acad\u00e9mico, haber completado la tutor\u00eda obligatoria previa a la matr\u00edcula.\n\n4. Si el estudiante ha dejado de estudiar, reservado matr\u00edcula o solicitado retiro total en el \u00faltimo per\u00edodo, deber\u00e1 tramitar previamente su reincorporaci\u00f3n a trav\u00e9s de la plataforma intranet-alumnos.\n\nProcedimiento para la matr\u00edcula regular:\n1. Generar la orden de pago del autoseguro estudiantil a trav\u00e9s de la plataforma INTRALU. Para ello, haz clic en \"Tr\u00e1mites\" para acceder al m\u00f3dulo de tr\u00e1mites y pagos. Luego, selecciona el tr\u00e1mite correspondiente, haz clic en \"Nuevo\" y sigue las instrucciones para completar el proceso.\n2. Realizar el pago del autoseguro dentro del plazo establecido en el calendario de actividades acad\u00e9micas, acerc\u00e1ndose a un sucursal del BCP o a traves de la aplicaci\u00f3n movil del banco. No se requiere la entrega del voucher de pago del autoseguro ni otros documentos.\n3. En la fecha y turno asignado, el estudiante llevar\u00e1 a cabo a su matr\u00edcula de forma virtual, es decir, realizar\u00e1 la inscripci\u00f3n de sus cursos a trav\u00e9s de la plataforma intranet-alumnos. Acceder\u00e1 al m\u00f3dulo de Matr\u00edcula UNI dentro de esta plataforma, seleccionar\u00e1 las asignaturas y horarios deseados, y finalizar\u00e1 confirmando la matr\u00edcula en todas las asignaturas elegidas en el sistema.\n\nLos turnos de matr\u00edcula se asignan por grupos seg\u00fan el promedio ponderado de los dos \u00faltimos ciclos acad\u00e9micos, en orden de m\u00e9rito. Los estudiantes pueden consultarlos en el m\u00f3dulo de Matr\u00edcula UNI de la intranet-alumnos.",
                        "relatedness": 0.649982213973999
                    },
                    {
                        "text": "Matricula Ingresantes\nLos nuevos ingresantes a la Facultad de Ciencias de la Universidad Nacional de Ingenieria, deberan seguir realizar el siguiente proceso para completar su matricula (primera matricula al primer ciclo):\n    1. Recabar su constancia de ingreso. Luego de gestionar la emisi\u00f3n de su constancia de ingreso, la Direcci\u00f3n de Admisi\u00f3n (DIAD) de la UNI enviar\u00e1 a su correo electr\u00f3nico la constancia de ingreso. Para m\u00e1s detalles sobre el proceso, puede consultar con DIAD al correo informes.admision@uni.edu.pe. \n    2. Actualizacion de datos en DIRCE. La Direcci\u00f3n de Registros Central y Estad\u00edstica har\u00e1 llegar a su correo su clave para acceder a la plataforma para intranet-alumnos y completar sus datos.\n    3. Registrar los datos en la Facultad de Ciencias. La oficina de estad\u00edstica enviar\u00e1 al correo del ingresante la ficha de datos. El llenado es car\u00e1cter obligatorio.\n    4. Efectuar el pago por autoseguro dentro del plazo establecido en el cronograma de actividades academicas publicado en la pagina web de la Facultad de ciencias. Para ello, el ingresante debera generar una orden de pago atravez del portal INTRALU, luego podra realizar el pago por aplicativo movil BCP o en alguna sucursal de la entidad bancaria.\n    5. Realizar la entrega de los siguientes documentos a la oficina de estad\u00edstica seg\u00fan el cronograma de actividades de matr\u00edcula de ingresantes publicado en la secci\u00f3n 'MATR\u00cdCULA Y PROCEDIMIENTOS' en la p\u00e1gina web de la Facultad de Ciencias.\n        - Contancia de Ingreso\n        - Ficha de datos Personales. Enviada por la oficina de estad\u00edstica al correo del ingresante.\n        - Constancia de Evaluacion Socioeconomica. El ingresante debera acudir a la cita para su evaluacion socioeconomica que le llegara a su correo. \n        - Certificado M\u00e9dico expedido por el Centro Medico UNI. Para ello acudir a su examen medico segun los cronograma publicados por la Direcci\u00f3n de Admisi\u00f3n. \n        - Comprobante de pago de Autoseguro Estudiantil\n    6. AERA ejecutar\u00e1 la matr\u00edcula (inscripci\u00f3n de cursos) de los ingresantes seg\u00fan el cronograma de actividades, \u00fanicamente para aquellos que hayan cumplido con la entrega de la entrega de los documentos requeridos en de las fechas establecidas en el cronograma.\n    7. Los ingresantes matriculados recibir\u00e1n un mensaje en su correo con el horario de sus cursos, ademas podran visualizar los cursos y horarios dentro del portal INTRALU.\n\nPara generar una orden de pago, el ingresante debe ingresar a la plataforma INTRALU. En esta plataforma, selecciona la opci\u00f3n \"Tr\u00e1mites\" para acceder al m\u00f3dulo de pagos. En dicho m\u00f3dulo, selecciona el tr\u00e1mite correspondiente en el men\u00fa desplegable, haz clic en el bot\u00f3n \"Nuevo\" y sigue las indicaciones hasta generar la orden de pago. Para mas detalles sobre el proceso para generar un orden de pogo puede revisar el manual de pagos de la UNI publicado en la pagina web de la Facultad de Ciencias.\n",
                        "relatedness": 0.6069541811943054
                    },
                    {
                        "text": "Traslado Interno Procedimiento Y Requisitos\n\u00bfC\u00f3mo es el procedimiento para que un estudiante de pregrado de la Facultad de Ciencias realice un traslado interno a otra carrera dentro de la misma Facultad?\nPara llevar a cabo un traslado interno, un estudiante de la Facultad de Ciencias deber\u00e1 seguir el siguiente procedimiento:\n1. Realizar el pago correspondiente por concepto de primer o segundo traslado interno, seg\u00fan sea el caso (S/ 125.20 o S/ 300.00). Para ello, el estudiante deber\u00e1 generar una orden de pago a trav\u00e9s de la plataforma INTRALU y efectuar el pago en una agencia del BCP o mediante el aplicativo m\u00f3vil del banco.\n2. El estudiante deber\u00e1 presentar su solicitud a trav\u00e9s de el plataforma intranet-alumnos, eligiendo la opci\u00f3n \"Traslado Interno\" en el men\u00fa \"Mis Tr\u00e1mites\". Esta solicitud deber\u00e1 ser enviada a traves de la plataforma en la fecha indicada en el calendario de actividades acad\u00e9micas, adjuntando el comprobante de pago por concepto de traslado interno y la ficha acad\u00e9mica.\n3. La Escuela verificar\u00e1 los documentos y requisitos propios dentro del primer d\u00eda \u00fatil de presentada la solicitud, registrar\u00e1 el expediente y remitir\u00e1 un correo de recepci\u00f3n al interesado.\n4. El director de Escuela o la Comisi\u00f3n de Matr\u00edcula analizar\u00e1n las solicitudes y, utilizando el formato disponible en la web DIRCE, inscribir\u00e1n las convalidaciones pertinentes seg\u00fan el Avance Curricular del alumno, las notas y las normas de Convalidaci\u00f3n.\n5. Con la autorizaci\u00f3n del director, el expediente ser\u00e1 enviado al Decano y posteriormente presentado ante el Consejo de Facultad para su aprobaci\u00f3n mediante Resoluci\u00f3n Decanal y remisi\u00f3n a la DIRCE.\n6. La DIRCE ejecutar\u00e1 el cambio de especialidad y realizar\u00e1 las convalidaciones.\n7. La Oficina de Estad\u00edstica gestionar\u00e1 la matr\u00edcula de los estudiantes por traslado interno en funci\u00f3n a la solicitud de matr\u00edcula del director de la Escuela Profesional, y finalmente, la DIRCE realizar\u00e1 la ejecuci\u00f3n de la matr\u00edcula en el sistema SIGA.\n8. No se recibir\u00e1n solicitudes extempor\u00e1neas.",
                        "relatedness": 0.6002129912376404
                    },
                    {
                        "text": "Procedimiento Para Solicitar Una Constancia De Matricula En La Facultad De Ciencias De La Uni\n\u00bfQu\u00e9 procedimiento debe seguir un estudiante para solicitar una constancia de Matricula en ingles o en espa\u00f1ol?\nPara solicitar una constancia de matr\u00edcula en ingl\u00e9s o en espa\u00f1ol, el estudiante deber seguir los siguientes pasos:\n\n1. Enviar un correo a estadistica_fc@uni.edu.pe solicitando una orden de pago para la CONSTANCIA DE MATR\u00cdCULA. En el mensaje, deber\u00e1 incluir sus datos personales: n\u00famero de DNI, apellidos, nombres, correo institucional y/o alternativo.\n\n2. La oficina de estad\u00edstica (AERA) le enviar\u00e1 la orden de pago por el monto correspondiente al correo electr\u00f3nico del estudiante.\n\n3. Realize el pago en alguna sucursal del BCP o a trav\u00e9s aplicaci\u00f3n movil del banco. En la app selecciona \"Pagar servicios\", elige la Universidad Nacional de Ingenier\u00eda, luego la opci\u00f3n de pago para estudiantes, e ingresa su n\u00famero de DNI. La app mostrar\u00e1 la orden de pago con el monto exacto para realizar el pago.\n\n4. Luego, deber\u00e1 enviar la solicitud correspondiente a mesa de partes de la facultad de ciencias al correo mesadepartes_fc@uni.edu.pe. Es importante que se asegure de indicar en la solicitud si desea la CONSTANCIA DE MATR\u00cdCULA en ingl\u00e9s o espa\u00f1ol. El modelo para esta solicitud est\u00e1 disponible dentro de la secci\u00f3n [\"MATR\u00cdCULA Y PROCEDIMIENTOS\" en la p\u00e1gina web de la Facultad de Ciencias](https://fc.uni.edu.pe/estadistica/documentos/).\n\n5. Por \u00faltimo, la constancia se enviar\u00e1 al correo institucional del estudiante.",
                        "relatedness": 0.5496701002120972
                    },
                    {
                        "text": "Proceso De Matricula\nDe los Requisitos de la Matr\u00edcula\nArt. 46\u00b0 La ense\u00f1anza en el antegrado es gratuita para la primera carrera profesional. Son requisitos para la matr\u00edcula, los siguientes:\n\ta. Cumplir con los requisitos acad\u00e9micos establecidos en el presente reglamento .\n\tb. Realizar el pago por concepto de Autoseguro M\u00e9dico. En caso de inconvenientes, presentar por correo o de forma f\u00edsica el recibo de pago.\n\tc. En caso de la matricula regular, reincorporaci\u00f3n o traslado interno, no presentar adeudos con su Facultad (econ\u00f3mico, libros, enseres, o material de ense\u00f1anza en general)\n\td. Aquellos estudiantes que no puedan cumplir con las deudas del tipo econ\u00f3mico podr\u00e1n solicitar, con diez (1O) d\u00edas \u00fatiles de anticipaci\u00f3n, facilidades a la Comisi\u00f3n de Matr\u00edcula y comprometerse a una forma de pago a plazos, evitando que sea una restricci\u00f3n de su matr\u00edcula. Esta ventaja se perder\u00e1 si no hubiese cumplido con similar compromiso en su anterior matr\u00edcula.\n\te. En el caso de los traslados internos, externos, graduados, titulados, convenios internacionales, adem\u00e1s de los incisos anteriores, deben presentar la Constancia o la Resoluci\u00f3n Directora! de convalidaci\u00f3n de las asignaturas. \n\tf. Los estudiantes que realizan su matr\u00edcula en forma rezagada deber\u00e1n efectuar el pago correspondiente establecido en las normas de la UNI. \n\tg. Los estudiantes que hayan dejado de estudiar, o reservado matr\u00edcula o hayan solicitado su retiro total en alg\u00fan periodo anterior, deber\u00e1n previamente haber solicitado su reincorporaci\u00f3n ante su Decano y su tr\u00e1mite lo realizar\u00e1 v\u00eda la plataforma DIRCE-UNI.\n\t\t Si requiriesen cambio previo de Plan de Estudios y convalidaciones correspondientes, \u00e9stas ya deben haberse tramitado. En estos casos , el tr\u00e1mite de reincorporaci\u00f3n debe hacerse con no menos de una semana adicional de anticipaci\u00f3n.\n\th. El pago del carn\u00e9 universitario no es requisito para la matr\u00edcula.\n\ti. No ser\u00e1 necesario que los estudiantes hayan participado en el \u00faltimo proceso electoral para ser matriculados, sin que esto signifique exonerarlo de la multa correspondiente. \n\tj. Otros de acuerdo con lo establecido en las normas de la UNI.",
                        "relatedness": 0.5412235915660858
                    },
                    {
                        "text": "Matricula Ingresantes\n                MATR\u00cdCULA DE INGRESANTES\n(del Reglamento de Matr\u00edcula Aprobado R.R. No0570 del 29.03.2022 y las modificacion de la Resoluci\u00f3n Rectoral N\u00b0 3084-2024-UNI)\n\nArt\u00edculo 7\u00b0: La matr\u00edcula de ingresantes es el procedimiento a trav\u00e9s del cual el ingresante acredita su condici\u00f3n de estudiante al inscribirse en un periodo acad\u00e9mico, en las fechas establecidas en el calendario de actividades acad\u00e9micas seg\u00fan la facultad que le corresponde, para matricularse no es obligatorio tomar el m\u00e1ximo de cr\u00e9ditos permitidos. La matr\u00edcula implica el cumplimiento de la Ley Universitaria y es el resultado de un acto formal que es ejecutado personal y voluntariamente por el estudiante, sujeto a verificaci\u00f3n posterior. Implica el cumplimiento de la Ley Universitaria, el Estatuto, las normas de la UNI y el presente reglamento.\n\n-----------------------------------------------------------------------------------------------------------------------------------------------\n\nProcedimiento de Matricula Para Ingresantes",
                        "relatedness": 0.531317925453186
                    },
                    {
                        "text": "Procedimiento Para Solicitar El Correo Institucional\n\u00bfQu\u00e9 procedimiento debe seguir un estudiante de la UNI para solicitar su correo institucional?\nPara solicitar su correo institucional de la UNI, enviar un correo obtenercorreo@uni.pe proporcionando la siguiente informaci\u00f3n:\n    C\u00f3digo de Alumno.\n    Nombres y Apellidos.\n    DNI.\n    Especialidad.\n    Indicar si es estudiante de pregrado o posgrado.\n    Correo personal (que no sea @uni.pe ni @UNI.PE) donde se le enviar\u00e1 la clave.\n    N\u00famero de Celular.\n    Facultad.",
                        "relatedness": 0.5192470550537109
                    },
                    {
                        "text": "Proceso De Matricula\n\tTodas las deudas pendientes del estudiante ser\u00e1n exigidas al finalizar sus estudios e iniciar el tr\u00e1mite para obtener de su grado de bachiller.\n\nDel Proceso de la Matr\u00edcula Regular para Estudiantes de Antegrado\n(del Reglamento de Matr\u00edcula Aprobado R.R. No0570 del 29.03.2022)\n\nArt. 47\u00b0 La matr\u00edcula regular se realiza v\u00eda internet a trav\u00e9s del Sistema de Matr\u00edcula UNI suministrado por DIRCE, en las fechas establecidas en el Calendario de Actividades de los Estudios de Antegrado de la UNI.\nEste proceso es personal y se realizar puede desde cualquier equipo con acceso a internet y ser\u00e1 bajo responsabilidad del estudiante, cumpliendo los siguientes pasos: \n\ta. Ingresar al Sistema de Matr\u00edcula UNI seg\u00fan el turno programado.\n\tb. En su Oferta de Matr\u00edcula, seleccionar las asignaturas-secci\u00f3n, comenzando por las obligatorias y de menor ciclo.\n\tc. Confirmar en el sistema la matr\u00edcula en las asignaturas-secci\u00f3n seleccionadas dentro del plazo establecido para dicho acto, consignando la secci\u00f3n elegida.\n\td. El sistema presentar\u00e1 el Reporte de Matr\u00edcula del estudiante para su impresi\u00f3n, remisi\u00f3n , o archivo.\n\te. En caso de los estudiantes que est\u00e1n en riesgo acad\u00e9mico y de los que retornen de la suspensi\u00f3n acad\u00e9mica realizar\u00e1n su matr\u00edcula, seg\u00fan lo coordinado con su respectivo tutor, quien autorizar\u00e1 la activaci\u00f3n de su c\u00f3digo y supervisar\u00e1 el respectivo Reporte.",
                        "relatedness": 0.4989213466644287
                    },
                    {
                        "text": "Traslado Interno\nDetalle del procedimiento de Traslado Interno: \n    1. Realizar el pago correspondiente por concepto de primer o segundo traslado interno, seg\u00fan sea el caso (S/ 125.20 o S/ 300.00). Para ello, el estudiante deber\u00e1 generar una orden de pago a trav\u00e9s de la plataforma INTRALU y efectuar el pago en una agencia del BCP o mediante el aplicativo m\u00f3vil del banco.\n    2. El interesado presentar\u00e1 su solicitud a trav\u00e9s del intranet alumnos / tramites; hasta la fecha que indica el calendario acad\u00e9mico. Adjuntando lo siguiente: \n        \u2212 Comprobante de pago por el concepto de Traslado Interno. \n        \u2212 Ficha Acad\u00e9mica \n    3. La Escuela, dentro del 1er d\u00eda \u00fatil de presentada verificar\u00e1 documentos y requisitos propios, registrar\u00e1 el expediente y remitir\u00e1 correo de recepci\u00f3n al interesado. Luego, si fuere necesario, solicitar\u00e1 a otras Escuelas Profesionales de la UNI la remisi\u00f3n de s\u00edlabos pertinentes, si el director lo solicita y atender\u00e1 similares solicitudes de otras Escuelas  \n    4. El director de Escuela o la Comisi\u00f3n de Matr\u00edcula analizar\u00e1 las solicitudes y haciendo uso del formato disponible en la web DIRCE, inscribir\u00e1 las convalidaciones pertinentes, seg\u00fan el Avance Curricular del alumno, las notas y las normas de Convalidaci\u00f3n. Si quedan notas no registradas, ser\u00e1n motivo de convalidaci\u00f3n posterior a la matr\u00edcula.  \n    5. Con  la  autorizaci\u00f3n  del  director,  el  expediente  es  enviado  al  Decano  y  posteriormente presentarlo ante el Consejo de Facultad para su aprobaci\u00f3n (mediante Resoluci\u00f3n Decanal) y remisi\u00f3n a la DIRCE.  \n    6. La DIRCE ejecutar\u00e1 el cambio de especialidad y/o facultad y las convalidaciones.   \n    7. Registradas las convalidaciones, la Oficina de Estad\u00edstica gestiona la matr\u00edcula de los sestudiantes por traslado interno en funci\u00f3n a la solicitud de matr\u00edcula del director de la Escuela Profesional, finalmente la DIRCE realiza la ejecuci\u00f3n de la matr\u00edcula en el sistema SIGA. \n    8. No se recibir\u00e1n solicitudes extempor\u00e1neas ",
                        "relatedness": 0.4829025506973267
                    }
                ]

print(context)